----
# Cuaderno 4 - División de datos y validación cruzada
## Ariel Palazzesi - 2026
----

En este cuaderno vamos a aprender:

- Cómo dividir el dataset en conjuntos de entrenamiento y prueba.
- Cómo entrenar un modelo simple y detectar posibles casos de *overfitting* o *underfitting*.
- Cómo aplicar validación cruzada para obtener una evaluación más robusta del modelo.

La idea es experimentar con los datos de forma práctica y reflexiva.


## Cómo usar el dataset

En este cuaderno usamos el archivo `Titanic-Dataset.csv`. Para cargarlo en Colab:

1. Hacé clic en el ícono de la carpeta en el margen izquierdo.
2. Subí el archivo `Titanic-Dataset.csv` desde tu computadora.
3. Verificá que aparezca en la carpeta `/content/`.

Luego, ejecutá el siguiente bloque de código para cargar los datos y comenzar a trabajar.


In [4]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

# Estilo general para los gráficos
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 5)

# Cargamos el dataset
# df = pd.read_csv("/content/Titanic-Dataset.csv")
df = pd.read_csv("Titanic-Dataset.csv")

# Mostramos las primeras filas
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Selección de variables para el modelo

Antes de entrenar cualquier modelo, es importante seleccionar qué columnas vamos a utilizar como **características predictoras** (también llamadas *features*) y cuál será nuestra **variable objetivo** o *target*.

En este ejemplo, vamos a predecir si una persona sobrevivió al hundimiento del Titanic. Por lo tanto, la variable objetivo será `Survived`, que contiene un 0 si no sobrevivió y un 1 si sí lo hizo.

Como características predictoras, vamos a usar algunas variables que ya fueron transformadas en cuadernos anteriores (por ejemplo, codificación de variables categóricas y escalado de numéricas). Para simplificar este ejemplo, seleccionaremos algunas de las más limpias y representativas:

- `Pclass_encoded`: clase del pasajero (ya codificada como 0, 1, 2)
- `Sex`: género del pasajero (vamos a codificarla a continuación)
- `Age_normalizada`: edad escalada
- `Fare_normalizada`: tarifa escalada
- `Embarked_C`, `Embarked_Q`, `Embarked_S`: variables dummy creadas a partir del puerto de embarque

> Si no tenés estas columnas listas desde los caudernosanteriores, no te preocupes: podés agregar solo algunas de ellas o adaptarlas con las transformaciones que ya hayas practicado.


In [6]:
# Para este ejemplo, creamos una copia del dataset original
df_modelo = df.copy()

# Codificamos la variable 'Sex' manualmente
df_modelo['Sex_encoded'] = df_modelo['Sex'].map({'male': 0, 'female': 1})

# Aplicamos One-Hot Encoding a 'Embarked' (si no fue hecho aún)
embarked_dummies = pd.get_dummies(df_modelo['Embarked'], prefix='Embarked', drop_first=False)
df_modelo = pd.concat([df_modelo, embarked_dummies], axis=1)

# Escalamos 'Age' y 'Fare' si no están escaladas
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_modelo[['Age_normalizada', 'Fare_normalizada']] = scaler.fit_transform(df_modelo[['Age', 'Fare']])

# Definimos X (features) e y (target)
columnas_features = ['Pclass', 'Sex_encoded', 'Age_normalizada', 'Fare_normalizada',
                     'Embarked_C', 'Embarked_Q', 'Embarked_S']
X = df_modelo[columnas_features]
y = df_modelo['Survived']

# Mostramos las primeras filas
X.head()


,Pclass,Sex_encoded,Age_normalizada,Fare_normalizada,Embarked_C,Embarked_Q,Embarked_S
0,3,0,0.271174,0.014151,False,False,True
1,1,1,0.472229,0.139136,True,False,False
2,3,1,0.321438,0.015469,False,False,True
3,1,1,0.434531,0.103644,False,False,True
4,3,0,0.434531,0.015713,False,False,True


## División en conjuntos de entrenamiento y prueba

Una vez que tenemos listas nuestras variables predictoras (`X`) y nuestra variable objetivo (`y`), el siguiente paso es dividir el dataset en dos partes:

- Un conjunto de **entrenamiento**, que se usará para ajustar el modelo.
- Un conjunto de **prueba**, que se usará para evaluar si el modelo funciona bien con datos que no vio durante el entrenamiento.

Esta división nos permite simular un entorno real: queremos que el modelo aprenda con un conjunto de datos, y luego comprobar si puede generalizar lo aprendido a nuevos casos.

Vamos a usar una división clásica: **80% para entrenamiento y 20% para prueba**, utilizando la función `train_test_split` de `scikit-learn`, que realiza esta separación de forma aleatoria pero reproducible si se fija una semilla (`random_state`).


## Eliminación de valores faltantes

Antes de entrenar nuestro modelo, debemos asegurarnos de que no haya valores faltantes (NaN) en las columnas que vamos a utilizar. Muchos algoritmos de `scikit-learn`, como la `Regresión Logística`, no pueden trabajar con datos incompletos.

En este caso, algunas variables como `Age` o `Embarked` pueden tener valores vacíos. Para resolverlo de forma sencilla, vamos a eliminar todas las filas que tengan al menos un valor faltante en las columnas seleccionadas para el modelo.

En cuadernos futuros veremos otras técnicas más avanzadas para tratar datos faltantes, como la imputación automática o el uso de modelos que aceptan NaN.


In [7]:
# Combinamos X e y para eliminar filas incompletas
df_modelo_clean = pd.concat([X, y], axis=1)

# Eliminamos filas con valores NaN
df_modelo_clean = df_modelo_clean.dropna()

# Separamos nuevamente X e y
X = df_modelo_clean[columnas_features]
y = df_modelo_clean['Survived']

# Verificamos que no queden valores faltantes
print("¿Hay NaN en X?", X.isna().sum().sum() > 0)
print("¿Hay NaN en y?", y.isna().sum() > 0)


¿Hay NaN en X? False
¿Hay NaN en y? False


In [8]:
# Dividimos los datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Verificamos las formas de los conjuntos
print(f"Datos de entrenamiento: {X_train.shape}")
print(f"Datos de prueba: {X_test.shape}")


Datos de entrenamiento: (571, 7)
Datos de prueba: (143, 7)


## Entrenamiento de un modelo simple: Regresión Logística

Una vez que los datos fueron preparados y divididos en conjuntos de entrenamiento y prueba, podemos avanzar hacia el **entrenamiento de nuestro primer modelo de Machine Learning**.

En este caso utilizaremos **Regresión Logística**, un algoritmo clásico para problemas de **clasificación binaria**, es decir, situaciones donde la variable objetivo solo puede tomar dos valores posibles (por ejemplo, *sí/no*, *0/1*, *sobrevive/no sobrevive*).

> **NOTA:** Aunque su nombre contiene la palabra *regresión*, la regresión logística se utiliza para **clasificar**. El modelo aprende una relación entre las variables de entrada y la probabilidad de pertenecer a una de las dos clases, y luego decide la clase final a partir de esa probabilidad.

Este modelo es una buena elección para comenzar porque:

* es simple y rápido de entrenar,
* suele funcionar bien como línea base,
* y sus resultados son relativamente fáciles de interpretar.

En el código de la celda siguiente, se importa primero la clase `LogisticRegression` desde `sklearn.linear_model`, que permite crear el modelo, y la función `accuracy_score` desde `sklearn.metrics`, que se utilizará para evaluar su rendimiento.

Al crear el modelo con `LogisticRegression(max_iter=1000)`, el argumento `max_iter` indica la cantidad máxima de iteraciones que el algoritmo puede realizar durante el proceso de entrenamiento. Aumentar este valor ayuda a asegurar que el modelo encuentre una solución adecuada, especialmente cuando el dataset es más complejo.

Luego, el método `fit(X_train, y_train)` entrena el modelo utilizando los datos de entrenamiento. En este paso, el algoritmo ajusta sus parámetros internos para aprender la relación entre las variables predictoras (`X_train`) y la variable objetivo (`y_train`).

Una vez entrenado el modelo, se realizan predicciones tanto sobre el conjunto de entrenamiento como sobre el conjunto de prueba utilizando el método `predict()`. Esto permite obtener:

* `y_pred_train`: las predicciones del modelo sobre los datos con los que fue entrenado,
* `y_pred_test`: las predicciones sobre datos que el modelo no vio durante el entrenamiento.

Finalmente, se calcula la **precisión** (*accuracy*) en ambos conjuntos mediante la función `accuracy_score()`. Esta métrica indica qué proporción de predicciones fue correcta. Comparar la precisión en entrenamiento y en prueba es fundamental para evaluar cómo generaliza el modelo:

* si la precisión es muy alta en entrenamiento pero baja en prueba, puede haber **sobreajuste**,
* si ambas precisiones son bajas, puede haber **subajuste**,
* si ambas son similares y razonables, el modelo está generalizando correctamente.

El objetivo de este primer modelo no es lograr el mejor rendimiento posible, sino **entender el proceso completo de entrenamiento, predicción y evaluación**, que será la base para trabajar con modelos más complejos en etapas posteriores.


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Creamos y entrenamos el modelo
modelo = LogisticRegression(max_iter=1000)
modelo.fit(X_train, y_train)

# Predicciones
y_pred_train = modelo.predict(X_train)
y_pred_test = modelo.predict(X_test)

# Evaluamos la precisión en entrenamiento y prueba
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

print(f"Precisión en entrenamiento: {accuracy_train:.4f}")
print(f"Precisión en prueba: {accuracy_test:.4f}")


Precisión en entrenamiento: 0.8039
Precisión en prueba: 0.7552


## ¿Cómo interpretamos estos resultados?

La **precisión** es una métrica simple que indica el porcentaje de predicciones correctas realizadas por el modelo.

En este ejemplo, comparamos la precisión sobre el conjunto de entrenamiento y el conjunto de prueba. Si la precisión en entrenamiento es muy alta pero baja en prueba, es probable que estemos ante un caso de **sobreajuste**: el modelo aprendió demasiado los datos que ya conocía, pero no generaliza bien a casos nuevos.

Si la precisión es baja en ambos conjuntos, probablemente el modelo no está capturando bien los patrones, lo que indica un posible **subajuste**.

Y si la precisión es razonablemente buena y similar en ambos conjuntos, significa que el modelo está logrando un buen equilibrio: **aprende, pero no memoriza**.

> Podés experimentar con otras proporciones de train/test o con diferentes variables para ver cómo cambian los resultados.


## Validación cruzada con K-Fold

Hasta ahora evaluamos el modelo utilizando una única división de los datos en conjunto de entrenamiento y conjunto de prueba. Si bien este enfoque es sencillo y muy usado, tiene una limitación importante: **los resultados pueden variar según cómo se haya realizado esa división**. Una partición distinta de los datos podría dar una precisión mayor o menor, sin que el modelo haya cambiado realmente.

La **validación cruzada** permite reducir este problema y obtener una evaluación más confiable del modelo. La idea central es dividir el conjunto de datos en varios grupos llamados *pliegues* (*folds*) y entrenar el modelo varias veces. En cada iteración, uno de los pliegues se utiliza como conjunto de validación y los restantes se usan para entrenar el modelo. De esta forma, **todos los datos se usan tanto para entrenar como para evaluar**, pero en momentos distintos.

En el código se utiliza la función `cross_val_score()` de `sklearn.model_selection`. Esta función se encarga automáticamente de:

* dividir los datos en pliegues,
* entrenar el modelo múltiples veces,
* evaluar el rendimiento en cada iteración.

El primer argumento de `cross_val_score()` es el modelo que queremos evaluar (`modelo`). Luego se pasan las variables predictoras `X` y la variable objetivo `y`. El parámetro `cv=5` indica que se utilizará una validación cruzada de **5 pliegues (5-Fold)**, una configuración muy común en la práctica. Esto significa que el modelo se entrenará y evaluará cinco veces, cada vez usando un pliegue distinto como conjunto de validación.

El argumento `scoring='accuracy'` define la métrica que se utilizará para evaluar el modelo en cada pliegue. En este caso se utiliza la **precisión (accuracy)**, que mide la proporción de predicciones correctas.

El resultado de `cross_val_score()` se guarda en la variable `scores`, que contiene un arreglo con la precisión obtenida en cada uno de los pliegues. Mostrar estos valores permite observar cómo varía el rendimiento del modelo según el subconjunto de datos utilizado para validación.

Y al final se calcula la **precisión promedio** utilizando `scores.mean()`. Este valor resume el desempeño general del modelo y suele ser una medida más **robusta y estable** que la obtenida con una sola partición entrenamiento-prueba.

El objetivo de aplicar validación cruzada en este punto es obtener una evaluación más confiable del modelo y reducir la posibilidad de sacar conclusiones basadas en una división particular de los datos que haya sido favorable o desfavorable por casualidad.


In [10]:
from sklearn.model_selection import cross_val_score

# Aplicamos validación cruzada de 5 pliegues
scores = cross_val_score(modelo, X, y, cv=5, scoring='accuracy')

# Mostramos los resultados
print("Precisión en cada pliegue:")
print(scores)

print(f"\nPrecisión promedio (cross-validation): {scores.mean():.4f}")


Precisión en cada pliegue:
[0.72027972 0.83916084 0.76223776 0.75524476 0.81690141]

Precisión promedio (cross-validation): 0.7788


## ¿Qué nos dice la validación cruzada?

En los resultados anteriores, cada número representa la precisión del modelo en una partición distinta del dataset. El promedio de estas precisiones nos da una idea más confiable del rendimiento real del modelo, ya que no depende de una única división de los datos.

Si los valores de precisión son muy diferentes entre sí, eso puede indicar que el modelo es **sensible a los datos con los que se entrena**, lo cual es una señal de alerta.

Si, en cambio, los valores son parecidos y estables, podemos confiar más en la evaluación y utilizar ese promedio como una referencia sólida para comparar modelos en el futuro.

Este tipo de evaluación más robusta será fundamental más adelante, cuando empecemos a construir, comparar y ajustar distintos algoritmos de Machine Learning.


## Conclusión y cierre

En este cuaderno recorrimos uno de los pasos más importantes en cualquier proyecto de Machine Learning: **evaluar correctamente el rendimiento del modelo**. Aprendimos a dividir los datos en conjuntos de entrenamiento y prueba, y vimos cómo aplicar **validación cruzada** para obtener resultados más robustos y confiables.

También entrenamos un modelo simple, observamos su comportamiento sobre diferentes conjuntos de datos, y detectamos cómo la forma en que dividimos la información puede influir en la interpretación del rendimiento.

Este enfoque nos ayuda a evitar errores comunes como el *overfitting* (cuando el modelo aprende demasiado y no generaliza) o el *underfitting* (cuando el modelo no logra captar los patrones relevantes).

> Recordá que entrenar un modelo es importante, pero saber **cómo y con qué criterio evaluarlo** es lo que realmente te convierte en una persona capaz de construir soluciones confiables.

¡Nos vemos en el próximo cuaderno!